In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None) # ver todas las filas

In [12]:
df1 = pd.read_csv("StudentsPerformance.csv")
df2 = pd.read_csv("xAPI-Edu-Data.csv")
df3 = pd.read_excel("Emotions.xlsx")
df4 = pd.read_csv("dataset_emociones_infancia_adolescencia.csv")

In [6]:
# Funcion para realizar un EDA Básico
def eda_basico(df):
    # 🌷 Mostrar ejemplos de datos para tener una primera idea
    print('🌷 Ejemplo de datos del DF:')
    display(df.head(3))    # Primeras 3 filas
    display(df.tail(3))    # Últimas 3 filas
    display(df.sample(3))  # 3 filas aleatorias
    print('─' * 100)

    # 🌻 Número total de filas (observaciones)
    print('🌻 Número de filas:')
    print(df.shape[0])
    print('─' * 100)

    # 🌱 Número total de columnas (variables)
    print('🌱 Número de columnas:')
    print(df.shape[1])
    print('─' * 100)

    # 🌼 Información general del DataFrame (tipos, nulos, etc.)
    print('🌼 Información de la tabla:')
    display(df.info())
    print('─' * 100)

    # 🌑 Lista con los nombres de las columnas
    print('🌑 Nombre de las columnas:')
    print(df.columns.tolist())
    print('─' * 100)

    # 🍄 Estadísticas descriptivas de columnas numéricas
    print('🍄 Descripción de los datos numéricos:')
    display(df.describe().T)
    print('─' * 100)

    # 🌋 Estadísticas descriptivas de columnas categóricas (si existen)
    print('🌋 Descripción de los datos no-numéricos:')
    if not df.select_dtypes(include='object').empty:
        display(df.describe(include='object').T)
    else:
        print('📭 No hay columnas tipo "object" en este DataFrame.')
    print('─' * 100)

    # 🍂 Cuántos valores únicos hay en cada columna
    print('🍂 Número de valores únicos por columna:')
    display(df.nunique())
    print('─' * 100)

    # 🐖 Cuántos valores nulos hay en cada columna
    print('🐖 Valores nulos por columna:')
    display(df.isnull().sum())
    print('─' * 100)

    # 🐲 Revisión de filas duplicadas
    print('🐲 Filas duplicadas:')
    duplicados = df.duplicated().sum()
    if duplicados > 0:
        print(f'📛 Hay {duplicados} filas duplicadas.')
        display(df[df.duplicated()].head(3))
    else:
        print('✅ No hay filas duplicadas.')
    print('─' * 100)

    # 🪹 Columnas constantes (con solo un valor único)
    print('🪹 Columnas constantes (solo un valor):')
    constantes = df.columns[df.nunique() <= 1]
    if not constantes.empty:
        print(f'📌 {len(constantes)} columnas con solo un valor:')
        display(constantes.tolist())
    else:
        print('✅ No hay columnas constantes.')
    print('─' * 100)

    # 🚀 Ver valores únicos de columnas categóricas (si existen)
    print('🚀 Valores únicos en columnas categóricas:')
    cat_cols = df.select_dtypes(include='object')
    if not cat_cols.empty:
        for col in cat_cols.columns:
            print(f'🔸 {col}')
            print(cat_cols[col].unique())
            print('─' * 60)
    else:
        print('📭 No hay columnas categóricas.')
    print('─' * 100)

    # 🧬 Resumen de tipos de datos presentes en el DataFrame
    print('🧬 Tipos de datos por columna:')
    display(df.dtypes.value_counts())
    print('─' * 100)

In [7]:
eda_basico(df1)

🌷 Ejemplo de datos del DF:


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77
999,female,group D,some college,free/reduced,none,77,86,86


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
22,male,group D,some college,standard,none,44,54,53
321,female,group E,high school,standard,none,75,86,79
85,female,group C,some college,standard,none,73,80,82


────────────────────────────────────────────────────────────────────────────────────────────────────
🌻 Número de filas:
1000
────────────────────────────────────────────────────────────────────────────────────────────────────
🌱 Número de columnas:
8
────────────────────────────────────────────────────────────────────────────────────────────────────
🌼 Información de la tabla:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7  

...

────────────────────────────────────────────────────────────────────────────────────────────────────
🐖 Valores nulos por columna:


gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

────────────────────────────────────────────────────────────────────────────────────────────────────
🐲 Filas duplicadas:
✅ No hay filas duplicadas.
────────────────────────────────────────────────────────────────────────────────────────────────────
🪹 Columnas constantes (solo un valor):
✅ No hay columnas constantes.
────────────────────────────────────────────────────────────────────────────────────────────────────
🚀 Valores únicos en columnas categóricas:
🔸 gender
['female' 'male']
────────────────────────────────────────────────────────────
🔸 race/ethnicity
['group B' 'group C' 'group A' 'group D' 'group E']
────────────────────────────────────────────────────────────
🔸 parental level of education
["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
────────────────────────────────────────────────────────────
🔸 lunch
['standard' 'free/reduced']
────────────────────────────────────────────────────────────
🔸 test preparation cour

object    5
int64     3
Name: count, dtype: int64

────────────────────────────────────────────────────────────────────────────────────────────────────


In [8]:
eda_basico(df2)

🌷 Ejemplo de datos del DF:


,Gender,Nationality,Place of birth,Educational Stages,Grade Levels,Section ID,Topic,Semester,Parent responsible for student,Raised hand,Visited resources,Viewing announcements,Discussion groups,Parent Answering Survey,ParentschoolSatisfaction,StudentAbsenceDays,Total
0,Male,KuwaIT,KuwaIT,lowerlevel,G-04,A,IT,First,Father,15,16,2,20,Yes,Good,Under-7,Middle-Level
1,Male,KuwaIT,KuwaIT,lowerlevel,G-04,A,IT,First,Father,20,20,3,25,Yes,Good,Under-7,Middle-Level
2,Male,KuwaIT,KuwaIT,lowerlevel,G-04,A,IT,First,Father,10,7,0,30,No,Bad,Above-7,Low-Level


,Gender,Nationality,Place of birth,Educational Stages,Grade Levels,Section ID,Topic,Semester,Parent responsible for student,Raised hand,Visited resources,Viewing announcements,Discussion groups,Parent Answering Survey,ParentschoolSatisfaction,StudentAbsenceDays,Total
477,Female,Jordan,Jordan,MiddleSchool,G-08,A,Geology,Second,Father,55,74,25,29,No,Bad,Under-7,Middle-Level
478,Female,Jordan,Jordan,MiddleSchool,G-08,A,History,First,Father,30,17,14,57,No,Bad,Above-7,Low-Level
479,Female,Jordan,Jordan,MiddleSchool,G-08,A,History,Second,Father,35,14,23,62,No,Bad,Above-7,Low-Level


,Gender,Nationality,Place of birth,Educational Stages,Grade Levels,Section ID,Topic,Semester,Parent responsible for student,Raised hand,Visited resources,Viewing announcements,Discussion groups,Parent Answering Survey,ParentschoolSatisfaction,StudentAbsenceDays,Total
56,Male,USA,USA,MiddleSchool,G-08,B,Math,First,Father,19,5,4,1,Yes,Good,Above-7,Low-Level
103,"Male,",KuwaIT,KuwaIT,lowerlevel,G-02,B,IT,First,Father,1,7,6,10,No,Bad,Above-7,Low-Level
346,Female,Jordan,Jordan,lowerlevel,G-02,B,French,Second,mom,24,97,15,14,No,Good,Under-7,High-Level


────────────────────────────────────────────────────────────────────────────────────────────────────
🌻 Número de filas:
480
────────────────────────────────────────────────────────────────────────────────────────────────────
🌱 Número de columnas:
17
────────────────────────────────────────────────────────────────────────────────────────────────────
🌼 Información de la tabla:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Gender                          480 non-null    object
 1   Nationality                     480 non-null    object
 2   Place of birth                  480 non-null    object
 3   Educational Stages              480 non-null    object
 4   Grade Levels                    480 non-null    object
 5   Section ID                      480 non-null    object
 6   Topic                           

...

────────────────────────────────────────────────────────────────────────────────────────────────────
🐲 Filas duplicadas:
📛 Hay 2 filas duplicadas.


,Gender,Nationality,Place of birth,Educational Stages,Grade Levels,Section ID,Topic,Semester,Parent responsible for student,Raised hand,Visited resources,Viewing announcements,Discussion groups,Parent Answering Survey,ParentschoolSatisfaction,StudentAbsenceDays,Total
326,"Male,",Jordan,Jordan,lowerlevel,G-02,A,French,First,Father,10,15,10,21,No,Bad,Above-7,Low-Level
327,"Male,",Jordan,Jordan,lowerlevel,G-02,A,French,Second,Father,30,10,20,5,No,Bad,Above-7,Low-Level


────────────────────────────────────────────────────────────────────────────────────────────────────
🪹 Columnas constantes (solo un valor):
✅ No hay columnas constantes.
────────────────────────────────────────────────────────────────────────────────────────────────────
🚀 Valores únicos en columnas categóricas:
🔸 Gender
['Male' 'Female' 'Male,' 'Female,']
────────────────────────────────────────────────────────────
🔸 Nationality
['KuwaIT' 'lebanon' 'Egypt' 'SaudiArabia' 'USA' 'Jordan' 'venzuela' 'Iran'
 'Tunis' 'Morocco' 'Syria' 'Palestine' 'Iraq' 'Lybia']
────────────────────────────────────────────────────────────
🔸 Place of birth
['KuwaIT' 'lebanon' 'Egypt' 'SaudiArabia' 'USA' 'Jordan' 'venzuela' 'Iran'
 'Tunis' 'Morocco' 'Syria' 'Iraq' 'Palestine' 'Lybia']
────────────────────────────────────────────────────────────
🔸 Educational Stages 
['lowerlevel' 'MiddleSchool' 'HighSchool']
────────────────────────────────────────────────────────────
🔸 Grade Levels
['G-04' 'G-07' 'G-08' 'G-06

object    13
int64      4
Name: count, dtype: int64

────────────────────────────────────────────────────────────────────────────────────────────────────


In [7]:
eda_basico(df3)

🌷 Ejemplo de datos del DF:


,Gender,Age,Grade,School_level,Tech_efficacy_1,Tech_efficacy_2,Tech_efficacy_3,Tech_efficacy_4,Tech_efficacy_5,Tech_efficacy_6,Tech_efficacy_7,Value_app_1,Value_app_2,Value_app_3,Value_app_4,Value_app_5,Control_app_1,Control_app_2,Control_app_3,Control_app_4,Control_app_5,Enjoyment1,Enjoyment2,Hopefulness1,Hopefulness2,Pride1,Pride2,Anxiety1,Anxiety2,Hopelessness1,Hopelessness2,Boredom1,Boredom2,Outcome_goal_1,Outcome_goal_2,Outcome_goal_3,Ability_goal_1,Ability_goal_2,Ability_goal_3,Neg_effort_1,Neg_effort_2,Neg_effort_3,Pos_effort_1,Pos_effort_2,Pos_effort_3,Tech_efficacy_aver,Control_app_aver,Value_app_aver,Enjoyment_aver,Hopefulness_aver,Pride_aver,Anger_aver,Anxiety_aver,Shame_aver,Hopelessness_aver,Boredom_aver,Outcome_goal_aver,Ability_goal_aver,Neg_effort_aver,Pos_effort_aver
0,0,11,4,1,5,5,5,5,4,4,5,4,5,5,4,5,5,4,5,4,5,5,4,4,5,2,2,2,2,2,2,2,2,5,5,4,4,5,5,4,5,4,4,4,5,4.50,4.5,4.6,4.5,4.5,2.0,2.0,2.0,2.0,2.0,2.0,4.666667,4.666667,4.333333,4.333333
1,0,11,4,1,5,5,5,5,5,4,5,5,4,4,4,4,4,5,5,4,5,4,4,4,4,4,5,2,2,2,2,2,2,5,5,5,5,5,5,3,5,2,5,5,5,4.75,4.5,4.2,4.0,4.0,4.5,4.5,2.0,2.0,2.0,2.0,5.000000,5.000000,3.333333,5.000000
2,0,10,4,1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,2,2,2,2,2,1,4,5,4,4,4,4,4,2,3,4,4,4,4.00,4.0,4.0,4.0,4.0,4.0,2.0,2.0,2.5,2.0,1.5,4.333333,4.000000,3.000000,4.000000


,Gender,Age,Grade,School_level,Tech_efficacy_1,Tech_efficacy_2,Tech_efficacy_3,Tech_efficacy_4,Tech_efficacy_5,Tech_efficacy_6,Tech_efficacy_7,Value_app_1,Value_app_2,Value_app_3,Value_app_4,Value_app_5,Control_app_1,Control_app_2,Control_app_3,Control_app_4,Control_app_5,Enjoyment1,Enjoyment2,Hopefulness1,Hopefulness2,Pride1,Pride2,Anxiety1,Anxiety2,Hopelessness1,Hopelessness2,Boredom1,Boredom2,Outcome_goal_1,Outcome_goal_2,Outcome_goal_3,Ability_goal_1,Ability_goal_2,Ability_goal_3,Neg_effort_1,Neg_effort_2,Neg_effort_3,Pos_effort_1,Pos_effort_2,Pos_effort_3,Tech_efficacy_aver,Control_app_aver,Value_app_aver,Enjoyment_aver,Hopefulness_aver,Pride_aver,Anger_aver,Anxiety_aver,Shame_aver,Hopelessness_aver,Boredom_aver,Outcome_goal_aver,Ability_goal_aver,Neg_effort_aver,Pos_effort_aver
2937,0,14,9,3,4,5,5,5,5,4,4,4,4,4,5,4,4,4,3,4,2,4,4,4,4,4,4,2,2,2,2,2,2,4,5,4,5,5,5,1,2,3,4,4,4,4.5,3.5,4.2,4.0,4.0,4.0,2.0,2.0,2.0,2.0,2.0,4.333333,5.0,2.000000,4.000000
2938,0,14,9,3,3,4,4,4,4,4,4,3,4,4,4,3,3,4,4,4,3,3,3,4,3,4,4,3,1,1,1,1,1,3,3,3,3,3,3,2,2,3,3,3,3,4.0,3.5,3.6,3.0,3.5,4.0,1.0,2.0,1.0,1.0,1.0,3.000000,3.0,2.333333,3.000000
2939,1,14,9,3,5,5,5,3,5,5,5,5,5,5,5,4,5,5,5,5,5,5,5,5,5,5,5,2,2,1,2,1,2,5,5,5,5,2,2,1,2,1,4,4,5,4.5,5.0,4.8,5.0,5.0,5.0,5.0,2.0,2.0,1.5,1.5,5.000000,3.0,1.333333,4.333333


,Gender,Age,Grade,School_level,Tech_efficacy_1,Tech_efficacy_2,Tech_efficacy_3,Tech_efficacy_4,Tech_efficacy_5,Tech_efficacy_6,Tech_efficacy_7,Value_app_1,Value_app_2,Value_app_3,Value_app_4,Value_app_5,Control_app_1,Control_app_2,Control_app_3,Control_app_4,Control_app_5,Enjoyment1,Enjoyment2,Hopefulness1,Hopefulness2,Pride1,Pride2,Anxiety1,Anxiety2,Hopelessness1,Hopelessness2,Boredom1,Boredom2,Outcome_goal_1,Outcome_goal_2,Outcome_goal_3,Ability_goal_1,Ability_goal_2,Ability_goal_3,Neg_effort_1,Neg_effort_2,Neg_effort_3,Pos_effort_1,Pos_effort_2,Pos_effort_3,Tech_efficacy_aver,Control_app_aver,Value_app_aver,Enjoyment_aver,Hopefulness_aver,Pride_aver,Anger_aver,Anxiety_aver,Shame_aver,Hopelessness_aver,Boredom_aver,Outcome_goal_aver,Ability_goal_aver,Neg_effort_aver,Pos_effort_aver
2410,0,13,8,3,5,5,5,5,5,4,4,4,4,5,5,4,4,4,5,5,3,4,5,5,5,5,5,1,2,1,1,1,1,5,5,5,2,5,4,1,2,3,4,3,5,4.50,4.0,4.4,4.5,5.0,5.0,1.0,1.5,1.5,1.0,1.0,5.000000,3.666667,2.000000,4.000000
2722,1,15,9,1,4,4,4,4,4,4,3,4,4,4,4,4,3,3,4,4,2,3,4,4,4,3,4,2,2,2,2,2,2,3,4,4,4,4,4,2,3,2,4,4,4,3.75,3.0,4.0,3.5,4.0,3.5,2.0,2.0,3.0,2.0,2.0,3.666667,4.000000,2.333333,4.000000
1674,0,12,6,3,4,5,5,5,5,4,4,5,4,4,5,4,5,5,4,3,3,4,4,5,5,5,5,1,1,1,1,1,1,5,5,5,5,5,5,5,5,4,5,4,4,4.50,4.0,4.4,4.0,5.0,5.0,1.0,1.0,1.5,1.0,1.0,5.000000,5.000000,4.666667,4.333333


────────────────────────────────────────────────────────────────────────────────────────────────────
🌻 Número de filas:
2940
────────────────────────────────────────────────────────────────────────────────────────────────────
🌱 Número de columnas:
60
────────────────────────────────────────────────────────────────────────────────────────────────────
🌼 Información de la tabla:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 60 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              2940 non-null   int64  
 1   Age                 2940 non-null   int64  
 2   Grade               2940 non-null   int64  
 3   School_level        2940 non-null   int64  
 4   Tech_efficacy_1     2940 non-null   int64  
 5   Tech_efficacy_2     2940 non-null   int64  
 6   Tech_efficacy_3     2940 non-null   int64  
 7   Tech_efficacy_4     2940 non-null   int64  
 8   Tech_efficacy_5    

...

────────────────────────────────────────────────────────────────────────────────────────────────────
🐲 Filas duplicadas:
📛 Hay 35 filas duplicadas.


,Gender,Age,Grade,School_level,Tech_efficacy_1,Tech_efficacy_2,Tech_efficacy_3,Tech_efficacy_4,Tech_efficacy_5,Tech_efficacy_6,Tech_efficacy_7,Value_app_1,Value_app_2,Value_app_3,Value_app_4,Value_app_5,Control_app_1,Control_app_2,Control_app_3,Control_app_4,Control_app_5,Enjoyment1,Enjoyment2,Hopefulness1,Hopefulness2,Pride1,Pride2,Anxiety1,Anxiety2,Hopelessness1,Hopelessness2,Boredom1,Boredom2,Outcome_goal_1,Outcome_goal_2,Outcome_goal_3,Ability_goal_1,Ability_goal_2,Ability_goal_3,Neg_effort_1,Neg_effort_2,Neg_effort_3,Pos_effort_1,Pos_effort_2,Pos_effort_3,Tech_efficacy_aver,Control_app_aver,Value_app_aver,Enjoyment_aver,Hopefulness_aver,Pride_aver,Anger_aver,Anxiety_aver,Shame_aver,Hopelessness_aver,Boredom_aver,Outcome_goal_aver,Ability_goal_aver,Neg_effort_aver,Pos_effort_aver
463,1,10,4,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,1,1,5,5,5,5,5,5,1,1,5,5,5,5,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,2.333333,5.0
477,0,10,4,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,1,1,5,5,5,5,5,5,1,1,5,5,5,5,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,2.333333,5.0
718,0,10,5,2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,1,1,5,5,5,5,5,5,1,1,1,5,5,5,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,3.0,1.0,1.0,5.0,5.0,1.000000,5.0


────────────────────────────────────────────────────────────────────────────────────────────────────
🪹 Columnas constantes (solo un valor):
✅ No hay columnas constantes.
────────────────────────────────────────────────────────────────────────────────────────────────────
🚀 Valores únicos en columnas categóricas:
📭 No hay columnas categóricas.
────────────────────────────────────────────────────────────────────────────────────────────────────
🧬 Tipos de datos por columna:


int64      45
float64    15
Name: count, dtype: int64

────────────────────────────────────────────────────────────────────────────────────────────────────


In [13]:
eda_basico(df4)

🌷 Ejemplo de datos del DF:


,edad,género,nivel_autocontrol,nivel_empático,habilidad_regulación,autoestima,autoconciencia,regulación_estrés,ansiedad,motivación_académica,rendimiento,satisfacción_escolar,apoyo_familiar,relaciones_sociales,atención_clase,bienestar_general
0,12,Niña,4.89,3.36,6.86,6.83,6.41,6.63,8.84,9.40,5.88,5.13,7.57,6.64,5.54,6.70
1,9,Niño,4.04,6.09,8.77,7.23,9.56,5.53,3.46,5.71,6.05,8.97,10.00,7.75,9.30,7.59
2,16,Niña,8.11,6.70,4.23,4.76,6.52,4.18,3.91,7.86,6.09,6.80,9.41,7.47,3.99,6.50


,edad,género,nivel_autocontrol,nivel_empático,habilidad_regulación,autoestima,autoconciencia,regulación_estrés,ansiedad,motivación_académica,rendimiento,satisfacción_escolar,apoyo_familiar,relaciones_sociales,atención_clase,bienestar_general
1197,9,Niña,3.38,6.65,5.83,7.43,6.27,5.51,2.30,8.07,6.26,7.54,7.01,8.12,7.70,6.60
1198,10,Niño,7.16,8.29,4.85,5.75,6.03,4.62,3.14,9.25,7.22,6.13,7.45,6.61,6.13,6.37
1199,13,Niña,3.91,7.39,3.39,5.97,6.90,4.37,4.15,7.66,6.04,5.89,7.09,8.73,4.96,6.04


,edad,género,nivel_autocontrol,nivel_empático,habilidad_regulación,autoestima,autoconciencia,regulación_estrés,ansiedad,motivación_académica,rendimiento,satisfacción_escolar,apoyo_familiar,relaciones_sociales,atención_clase,bienestar_general
445,10,Niño,6.55,6.50,4.85,8.13,7.16,7.44,5.35,8.02,7.44,5.53,7.07,7.72,7.34,7.01
358,8,Niña,5.23,5.71,9.19,5.94,5.90,4.53,4.56,6.26,6.82,5.90,7.68,6.94,2.32,6.35
1114,12,Niño,8.08,3.09,7.08,6.77,6.34,5.50,5.13,5.92,8.52,5.48,10.00,6.49,6.98,6.82


────────────────────────────────────────────────────────────────────────────────────────────────────
🌻 Número de filas:
1200
────────────────────────────────────────────────────────────────────────────────────────────────────
🌱 Número de columnas:
16
────────────────────────────────────────────────────────────────────────────────────────────────────
🌼 Información de la tabla:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   edad                  1200 non-null   int64  
 1   género                1200 non-null   object 
 2   nivel_autocontrol     1200 non-null   float64
 3   nivel_empático        1200 non-null   float64
 4   habilidad_regulación  1200 non-null   float64
 5   autoestima            1200 non-null   float64
 6   autoconciencia        1200 non-null   float64
 7   regulación_estrés     1200 non-null   float64
 8  

...

────────────────────────────────────────────────────────────────────────────────────────────────────
🐖 Valores nulos por columna:


edad                    0
género                  0
nivel_autocontrol       0
nivel_empático          0
habilidad_regulación    0
autoestima              0
autoconciencia          0
regulación_estrés       0
ansiedad                0
motivación_académica    0
rendimiento             0
satisfacción_escolar    0
apoyo_familiar          0
relaciones_sociales     0
atención_clase          0
bienestar_general       0
dtype: int64

────────────────────────────────────────────────────────────────────────────────────────────────────
🐲 Filas duplicadas:
✅ No hay filas duplicadas.
────────────────────────────────────────────────────────────────────────────────────────────────────
🪹 Columnas constantes (solo un valor):
✅ No hay columnas constantes.
────────────────────────────────────────────────────────────────────────────────────────────────────
🚀 Valores únicos en columnas categóricas:
🔸 género
['Niña' 'Niño']
────────────────────────────────────────────────────────────
────────────────────────────────────────────────────────────────────────────────────────────────────
🧬 Tipos de datos por columna:


float64    14
int64       1
object      1
Name: count, dtype: int64

────────────────────────────────────────────────────────────────────────────────────────────────────
